<a href="https://colab.research.google.com/github/Subhitsha123/Algorithim/blob/main/Tzu_Chi_data_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [277]:
from google.colab import files
uploaded = files.upload()

Saving SGO Data Tzu Chi SEEN@Bukit Batok (Mar 2025)_21.05.2025 (1).xls to SGO Data Tzu Chi SEEN@Bukit Batok (Mar 2025)_21.05.2025 (1) (3).xls
Saving Member Attendances Report (cth.member.attendance.report) (2).csv to Member Attendances Report (cth.member.attendance.report) (2) (3).csv
Saving Members (cth.user).csv to Members (cth.user) (4).csv


In [278]:
import pandas as pd
activities_df = pd.read_csv("Member Attendances Report (cth.member.attendance.report) (2).csv")
members_df = pd.read_csv("Members (cth.user).csv")

In [279]:

# nly 'Physical' activities
physical_activities = activities_df[activities_df['AAP Domain'].str.lower() == 'physical']

# Count number of physical sessions
physical_count = physical_activities.groupby('Phone').size().reset_index(name='Physical_Activity_Count')

members_df['Contact Number'] = members_df['Contact Number'].astype(str)
physical_count['Phone'] = physical_count['Phone'].astype(str)

merged_df = pd.merge(members_df, physical_count, left_on='Contact Number', right_on='Phone', how='left')

merged_df['Physical_Activity_Count'] = merged_df['Physical_Activity_Count'].fillna(-1).astype(int)


In [280]:
sgo = pd.read_excel("SGO Data Tzu Chi SEEN@Bukit Batok (Mar 2025)_21.05.2025 (1).xls", engine='xlrd')

In [281]:

activities_df['Session Detail/Date'] = pd.to_datetime(activities_df['Session Detail/Date'], errors='coerce')

# Group by phone and get last session date
last_session = (
    activities_df
    .groupby('Phone')['Session Detail/Date']
    .max()
    .reset_index()
    .rename(columns={'Session Detail/Date': 'Last_Activity_Date'})
)


from datetime import datetime
last_session['Activity_Days_Inactive'] = (datetime.now() - last_session['Last_Activity_Date']).dt.days


final_df = pd.merge(members_df, sgo, on='NRIC', how='left')

final_df['Contact Number'] = pd.to_numeric(final_df['Contact Number'], errors='coerce').fillna(0).astype('int64')
last_session['Phone'] = pd.to_numeric(last_session['Phone'], errors='coerce').fillna(0).astype('int64')

final_df = pd.merge(
    final_df,
    last_session[['Phone', 'Activity_Days_Inactive']],
    left_on='Contact Number',
    right_on='Phone',
    how='left'
)

final_df['Activity_Days_Inactive'] = final_df['Activity_Days_Inactive'].fillna(999).astype(int)


In [282]:
final_df.head(10)

,Age,Centre (as client),CFS_x,Contact Number,Full Name,Is Client,Is interested to volunteer for the centre?,Is NOK?,Services,Within Boundary,...,Indicate if senior is a care recipient or a main caregiver,Family or close friends report feeling overwhelmed by senior's illness.,"Living situation:Senior is a caregiver looking after someone who require high care needs (e.g. elderly who is main caregiver to someone who is ADL dependent, with behavioural issues, etc.)",Indication of low mood,Have you thought of harming yourself in the last 3 months?,Degree of loneliness,Has limited or no social interaction,What programmes would senior be interested in or want to see?,Phone,Activity_Days_Inactive
0,68.0,Tzu Chi SEEN @ Bukit Batok,1-3,91713243,Lim Guat Ngoo,No,No,No,Activities,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999
1,74.0,Tzu Chi SEEN @ Bukit Batok,1-3,97527335,Tng Tian Choo,No,No,No,Activities,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999
2,70.0,Tzu Chi SEEN @ Bukit Batok,1-3,90128201,Ng Suat Hiong,No,No,No,Activities,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999
3,73.0,Tzu Chi SEEN @ Bukit Batok,1-3,97442856,Lim Bee Hong,No,No,No,Activities,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97442856.0,36
4,68.0,Tzu Chi SEEN @ Bukit Batok,1-3,97562973,Tan Suan Ngar,No,No,No,Activities,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999
5,81.0,Tzu Chi SEEN @ Bukit Batok,NaN,83997090,Yap Soi Moy,No,No,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999
6,67.0,Tzu Chi SEEN @ Bukit Batok,NaN,98634572,Sng Lin Lin,No,No,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999
7,70.0,Tzu Chi SEEN @ Bukit Batok,NaN,96707945,Lee Guat Ngo,No,No,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96707945.0,0
8,58.0,Tzu Chi SEEN @ Bukit Batok,NaN,96570819,Ang Siew Hong Germain,No,Yes,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96570819.0,1
9,80.0,Tzu Chi SEEN @ Bukit Batok,NaN,96349582,Wee Shin,No,No,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999


In [283]:
final_df = final_df[final_df['Centre (as client)'] == 'Tzu Chi SEEN @ Bukit Batok']
final_df = final_df[final_df['Within Boundary'] == 'Yes']

In [284]:
final_df = final_df[final_df['Full Name'].notna() & (final_df['Full Name'].str.strip() != '')]


In [285]:
# def categorize_risk(row):
#     cfs = pd.to_numeric(row.get('CFS_x'), errors='coerce')
#     srf_raw = str(row.get('Social Risk Factor(s)', '')).strip()
#     if srf_raw == '>1':
#         srf = 2
#     else:
#         srf = pd.to_numeric(srf_raw, errors='coerce')

#     housing = str(row.get('Housing Type', '')).lower()
#     days_inactive = pd.to_numeric(row.get('Activity_Days_Inactive'), errors='coerce')
#     marital_status = str(row.get('Marital Status', '')).lower()
#     alone = any(x in marital_status for x in ['single', 'widowed', 'divorced'])

#     # New SGO factors
#     low_mood = str(row.get('Indication of low mood', '')).strip().lower() == 'yes'
#     suicidal_thoughts = str(row.get('Have you thought of harming yourself in the last 3 months?', '')).strip().lower() == 'yes'
#     loneliness = pd.to_numeric(row.get('Degree of loneliness'), errors='coerce')
#     social_cutoff = str(row.get('Has limited or no social interaction', '')).strip().lower() == 'yes'
#     unstable_condition = str(row.get('Instability of Conditions - conditions that make cognitive, ADL, mood or behaviour patterns unstable (fluctuating, precarious or deteriorating)', '')).strip().lower() == 'yes'

#     # Flags
#     frail = pd.notna(cfs) and cfs >= 6
#     moderate_frail = pd.notna(cfs) and cfs in [4, 5]
#     risky_srf = pd.notna(srf) and srf >= 1
#     poor_housing = any(x in housing for x in ['1-rm hdb', '2-rm hdb', '3-rm hdb'])
#     inactive = pd.notna(days_inactive) and days_inactive > 30
#     moderate_inactive = pd.notna(days_inactive) and days_inactive > 15

#     # R1: urgent cases
#     if (suicidal_thoughts or (loneliness >= 3 and low_mood and alone and frail)) and inactive:
#         return 'R1'

#     # R2: concerning
#     elif (frail or moderate_frail) and (low_mood or risky_srf or unstable_condition or social_cutoff) and (alone or poor_housing) and moderate_inactive:
#         return 'R2'

#     # R3: minor risk
#     elif (frail or moderate_frail or low_mood or risky_srf or social_cutoff or poor_housing) and inactive:
#         return 'R3'

#     # R4: low risk
#     else:
#         return 'R4'


In [297]:
import numpy as np
import pandas as pd

def categorize_risk(row):
    # --- CFS ---
    cfs_raw = str(row.get('CFS_x', '')).strip()
    if cfs_raw == '1-3':
        cfs = 2
    elif cfs_raw == '4':
        cfs = 4
    elif cfs_raw == '5':
        cfs = 5
    elif cfs_raw == '6':
        cfs = 6
    elif cfs_raw == '7-8':
        cfs = 7
    else:
        cfs = np.nan

    # --- Social Risk Factor ---
    srf_raw = str(row.get('Social Risk Factor(s)', '')).strip()
    if srf_raw == '>1':
        srf = 2
    else:
        srf = pd.to_numeric(srf_raw, errors='coerce')

    # --- Housing Type ---
    housing = str(row.get('Housing Type', '')).strip().lower()

    # --- Days Inactive ---
    days_inactive = pd.to_numeric(row.get('Activity_Days_Inactive'), errors='coerce')

    # --- Marital Status ---
    marital_status = str(row.get('Marital Status', '')).strip().lower()
    alone = any(x in marital_status for x in ['single', 'widowed', 'divorced'])

    # --- Low Mood ---
    mood_raw = str(row.get('Indication of low mood', '')).strip().lower()
    low_mood = mood_raw == 'yes' or 'could not' in mood_raw

    # --- Suicidal Thoughts ---
    suicidal_thoughts = str(row.get('Have you thought of harming yourself in the last 3 months?', '')).strip().lower() == 'yes'

    # --- Degree of Loneliness ---
    loneliness_raw = str(row.get('Degree of loneliness', '')).strip().lower()
    loneliness_map = {
        'not': 0,
        'occasionally': 1,
        'frequently': 2,
        'daily': 3
    }
    loneliness = next((v for k, v in loneliness_map.items() if loneliness_raw.startswith(k)), np.nan)

    # --- Social Cutoff ---
    social_cutoff = str(row.get('Has limited or no social interaction', '')).strip().lower() == 'yes'

    # --- Instability of Conditions ---
    unstable_condition = str(row.get('Instability of Conditions - conditions that make cognitive, ADL, mood or behaviour patterns unstable (fluctuating, precarious or deteriorating)', '')).strip().lower() == 'yes'

    # --- Flags ---
    frail = pd.notna(cfs) and cfs >= 6
    moderate_frail = pd.notna(cfs) and cfs in [4, 5]
    risky_srf = pd.notna(srf) and srf >= 1
    poor_housing = any(x in housing for x in ['1-rm', '2-rm', '3-rm'])
    inactive = pd.notna(days_inactive) and days_inactive > 30
    moderate_inactive = pd.notna(days_inactive) and days_inactive > 15

    # --- Risk Classification ---
    if (suicidal_thoughts or (pd.notna(loneliness) and loneliness >= 3 and low_mood and alone and frail)) and inactive:
        return 'R1'
    elif (frail or moderate_frail) and (low_mood or risky_srf or unstable_condition or social_cutoff) and (alone or poor_housing) and moderate_inactive and srf >= 1:
        return 'R2'
    elif (frail or moderate_frail or low_mood or risky_srf or social_cutoff or poor_housing) and moderate_inactive:
        return 'R3'
    elif any([frail, moderate_frail, low_mood, risky_srf, social_cutoff, poor_housing]):
      return 'R4'
    else:
        return 'Unclassified'


In [298]:
final_df['Risk Level'] = final_df.apply(categorize_risk, axis=1)


In [299]:
final_df['Risk Level'].value_counts().reindex(['R1', 'R2', 'R3', 'R4']).fillna(0).astype(int)


,count
Risk Level,
R1,0
R2,3
R3,369
R4,226


In [300]:
df2 = final_df[final_df['Risk Level'] == 'R2']
df2

,Age,Centre (as client),CFS_x,Contact Number,Full Name,Is Client,Is interested to volunteer for the centre?,Is NOK?,Services,Within Boundary,...,Family or close friends report feeling overwhelmed by senior's illness.,"Living situation:Senior is a caregiver looking after someone who require high care needs (e.g. elderly who is main caregiver to someone who is ADL dependent, with behavioural issues, etc.)",Indication of low mood,Have you thought of harming yourself in the last 3 months?,Degree of loneliness,Has limited or no social interaction,What programmes would senior be interested in or want to see?,Phone,Activity_Days_Inactive,Risk Level
1211,61.0,Tzu Chi SEEN @ Bukit Batok,4,85803461,Segeran S/O Rajoo Krishna,Yes,No,No,Activities,Yes,...,No,No,No,NaN,Frequently (weekly but less than daily),Yes,Senior does not have hobbies,85803461.0,48,R2
1641,64.0,Tzu Chi SEEN @ Bukit Batok,4,87122344,Jasman Bin Jalani,Yes,No,No,Activities,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87122344.0,53,R2
5150,67.0,Tzu Chi SEEN @ Bukit Batok,5,66986476,Rahim Bin Man,Yes,NaN,No,Activities,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66986476.0,34,R2


In [301]:
print(final_df.columns.tolist())

['Age', 'Centre (as client)', 'CFS_x', 'Contact Number', 'Full Name', 'Is Client', 'Is interested to volunteer for the centre?', 'Is NOK?', 'Services', 'Within Boundary', 'NRIC', 'Housing Type', 'Last F2F Date', 'Last Measured Blood Pressure', 'Last Participation Date', 'Last Measured Weight', 'Marital Status', 'Social Risk Factor(s)', 'Activity Type', 'Date of Assessment/Visit', 'Name of AAC', 'Name of Senior', 'UIN', 'Senior GRC', 'Senior Division', 'Year of Birth', 'Gender', 'Ethnicity', 'Language spoken/understood', 'Number of Risk Factor(s)', 'Social Support Recommendation', 'CFS_y', 'AAP Recommendation', 'Postal Code', 'Block', 'Street Name', 'Floor', 'Unit Number', 'Working Status', 'Senior Contact Number', 'Contact Number of Next of Kin', 'Title of Next of Kin', 'Name of Next of Kin', 'Preferred day to call/visit', 'Preferred time to call/visit', 'Short term memory - recall 3 unrelated items (e.g. watch, table, apple) after 5 mins.', 'Cognitive skills for daily decision making'

In [302]:
columns_to_check = [
    'CFS_x',
    'Social Risk Factor(s)',
    'Housing Type',
    'Activity_Days_Inactive',
    'Marital Status',
    'Indication of low mood',
    'Have you thought of harming yourself in the last 3 months?',
    'Degree of loneliness',
    'Has limited or no social interaction',
    'Instability of Conditions - conditions that make cognitive, ADL, mood or behaviour patterns unstable (fluctuating, precarious or deteriorating)'
]

for col in columns_to_check:
    print(f"\n--- Unique values in '{col}' ---")
    print(final_df[col].dropna().unique())



--- Unique values in 'CFS_x' ---
['1-3' '5' '4' '6' '7-8']

--- Unique values in 'Social Risk Factor(s)' ---
['0' '1' '>1']

--- Unique values in 'Housing Type' ---
['4-RM HDB' '2-RM HDB' '3-RM HDB' '5-RM HDB' 'EXECUTIVE CONDOMINIUM'
 'CONDOMINIUM' 'LANDED PROPERTY']

--- Unique values in 'Activity_Days_Inactive' ---
[999   1   5  18  55  25   9  44   2  27   0  53  11  16  14   8   6  46
  20  29  58  60  19  26   7  22  61  23  35  48  37  12  34   4  63  33
  32  69  13  68  62  40  57  42  21  15  67  28  49  36  70  51  47]

--- Unique values in 'Marital Status' ---
['MARRIED' 'PARTNER/SIGNIFICANT OTHER' 'WIDOWED' 'SINGLE' 'DIVORCED'
 'SEPARATED']

--- Unique values in 'Indication of low mood' ---
['No' 'Senior could not (would not) respond' 'Yes']

--- Unique values in 'Have you thought of harming yourself in the last 3 months?' ---
['No']

--- Unique values in 'Degree of loneliness' ---
['Not lonely' 'Senior could not (would not) respond'
 'Occasionally (once in a while or irre

In [303]:
relevant_columns = [
    'Full Name',
    'CFS_x',
    'Social Risk Factor(s)',
    'Housing Type',
    'Activity_Days_Inactive',
    'Marital Status',
    'Indication of low mood',
    'Have you thought of harming yourself in the last 3 months?',
    'Degree of loneliness',
    'Has limited or no social interaction',
    'Instability of Conditions - conditions that make cognitive, ADL, mood or behaviour patterns unstable (fluctuating, precarious or deteriorating)',
    'Risk Level'
]

display_df = final_df[relevant_columns]


In [306]:
display_df[display_df['Risk Level'] == 'R2'].head()


,Full Name,CFS_x,Social Risk Factor(s),Housing Type,Activity_Days_Inactive,Marital Status,Indication of low mood,Have you thought of harming yourself in the last 3 months?,Degree of loneliness,Has limited or no social interaction,"Instability of Conditions - conditions that make cognitive, ADL, mood or behaviour patterns unstable (fluctuating, precarious or deteriorating)",Risk Level
1211,Segeran S/O Rajoo Krishna,4,1,3-RM HDB,48,DIVORCED,No,NaN,Frequently (weekly but less than daily),Yes,No,R2
1641,Jasman Bin Jalani,4,1,3-RM HDB,53,WIDOWED,NaN,NaN,NaN,NaN,NaN,R2
5150,Rahim Bin Man,5,1,3-RM HDB,34,DIVORCED,NaN,NaN,NaN,NaN,NaN,R2
